# PSI

Over time  the ability to correctly predict probability of default (or correctly classify customers as good or bad) change with time due to drift. Drift (Shift) could arise as a result of different factors. Data being one of them.

Being able to track this drift is necessary, as a major drift or shift would require a complete remodelling of the score card.

**WHEN:**
* PSI < 0.1 - No change. You can continue using existing model.
* PSI >=0.1 but less than 0.2 - Slight change is required.
* PSI >=0.2 - Significant change is required. Ideally, you should not use this model any more.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
#from psi import calculate_psi

In [45]:
Q1 = pd.read_excel('psi 01-06 forin model vars.xlsx')
train=pd.read_excel('train psi.xlsx')


In [28]:
train.head(1)

,Unnamed: 0,Day_Diff_Debitgd,Investments_freq_2gd,Vol_ATM_Transactions_Count2gd,AddresLGAgd,Recurring_Debit_Count_2gd,avg_Monthly_cash_flowgd,IntlRemittancesFreq2gd,averageDebitgd,r_avCredit_avDebitgd,minimumCreditgd,GBFlag
0,111,"[3.0, 4.0)","[-0.0, 4.0)","[0.1, 3.0)",Others,"[111.0, 508.0)","[-4192.0, -1200.0)","[2.1, inf)","[376.0, 501.0)","[3.0, 5.0)","[176.0, inf)",Good


In [38]:
Q1['Investments_freq_2gd'].value_counts()

[0, 4.0)      10848
[4.0, inf)     2012
Name: Investments_freq_2gd, dtype: int64

In [40]:

train['Investments_freq_2gd'] = train['Investments_freq_2gd'].astype(str)
train['Investments_freq_2gd'] = np.where(train['Investments_freq_2gd'].isin(['[-0.0, 4.0)', '[-999.0, -0.0)']), '[0, 4.0)', train['Investments_freq_2gd'])

In [47]:
Q1.columns

Index(['Unnamed: 0', 'Day_Diff_Debitgd', 'avg_POSVolumegd',
       'Vol_ATM_Transactions_Count2gd', 'AddresLGAgd',
       'Recurring_Debit_Count_2gd', 'avg_Monthly_cash_flowgd',
       'IntlRemittancesFreq2gd', 'averageDebitgd', 'r_avCredit_avDebitgd',
       'minimumCreditgd', 'GBFlag'],
      dtype='object')

In [48]:
def PSI(Train_data, nonTrain_data, feature):
    dt = pd.DataFrame(Train_data[feature].value_counts())
    dt.rename(columns={feature:'Train'}, inplace=True)
    dt['Train %'] = dt['Train']/dt['Train'].sum()
    dt['2022 Q1'] = nonTrain_data[feature].value_counts()
    dt['2022 Q1 %'] = dt['2022 Q1']/dt['2022 Q1'].sum()
    dt['Diff'] = dt['Train %']- dt['2022 Q1 %']
    dt['Log'] = np.log(dt['Train %']/dt['2022 Q1 %'])
    dt['PSI'] = dt['Diff'] * dt['Log']
    dt['sum'] =dt['PSI'].sum()
    return dt

vars = [col for col in train.columns]
vars.remove('Unnamed: 0')


Q =Q1[vars] 
output = {}
for var in vars:
  
  #get PSI
  variable = PSI(train, Q, var)
  # append to dictionary

  output[var]=variable   


In [49]:
output['avg_POSVolumegd']

,Train,Train %,2022 Q1,2022 Q1 %,Diff,Log,PSI,sum
"[5.0, 25.0)",503,0.291594,3641,0.283126,0.008468,0.029471,0.000250,0.134497
"[25.0, 47.0)",349,0.202319,2009,0.156221,0.046098,0.258574,0.011920,0.134497
"[0.0, 5.0)",347,0.201159,4536,0.352722,-0.151562,-0.561581,0.085115,0.134497
"[67.0, inf)",347,0.201159,1742,0.135459,0.065701,0.395430,0.025980,0.134497
"[47.0, 67.0)",179,0.103768,932,0.072473,0.031295,0.358948,0.011233,0.134497


In [5]:
# def woe_iv(Train, Q, variablie_list):   
  
#     output = {}
#     for variable in variablie_list:
#         try:
#             var = PSI(Train, Q, variable)
            
#             for i in range(len(var.index.values)):
#                 output.setdefault('Variables',[]).append(variable)
#                 output.setdefault('Variable PSI',[]).append(var['sum'].values[i])
#         except Exception as e:
#             pass

#     return pd.DataFrame(output)
  
# dd= woe_iv(Train, Q, cols)

In [20]:
def Export_CA_Report(df, variablie_list, GBFlag, fileName='CA_Report'):
    
    # This inner function estimates WoE and IV which represent CA for all the provided variables
    def CA_Report(df, variablie_list, GBFlag):
        # Create a dictionary to track CA iteratively
        output = {}
        for variable in variablie_list:
            # use the function for estimating WoE for categorical variables to 
            var = woe_cat(df, variable, GBFlag)
            # append to dictionary
            output[variable]=var        

        return output
    
    
   # Instatiate above function to CA report for all provided variables
    ca_result = CA_Report(df, variablie_list, GBFlag)
    # save CA report to Excel
    with pd.ExcelWriter(fileName +'.xlsx',mode= 'a', engine="openpyxl", if_sheet_exists='overlay') as writer:
        for variable_name, data in ca_result.items():
            data.to_excel(writer, sheet_name=variable_name, startcol=14)

In [21]:
var =[col for col in train.columns]

Export_CA_Report(
    output, 
    var ,
    'GBFlag',
    )

NameError: name 'woe_cat' is not defined

In [18]:
output['total_credit_1gd']

[0.0, 94821.0)          0.30325
[94821.0, 189477.0)     0.30325
[189477.0, 380401.0)    0.30325
[558300.0, inf)         0.30325
[380401.0, 558300.0)    0.30325
Name: sum, dtype: float64

In [32]:
#append to excle

with pd.ExcelWriter('CA_Report for in-model vars-06-01.xlsx',mode= 'a', engine="openpyxl", if_sheet_exists='overlay') as writer:
        for variable_name, data in output.items():
            data.to_excel(writer, sheet_name=variable_name, startcol= 15, startrow=1)

In [ ]:
def PSI(Train_data, nonTrain_data, feature):
    dt = pd.DataFrame(Train_data[feature].value_counts())
    dt.rename(columns={feature:'Train'}, inplace=True)
    dt['Train %'] = dt['Train']/dt['Train'].sum()
    dt['2022 Q1'] = nonTrain_data[feature].value_counts()
    dt['2022 Q1 %'] = dt['2022 Q1']/dt['2022 Q1'].sum()
    dt['Diff'] = dt['Train %']- dt['2022 Q1 %']
    dt['Log'] = np.log(dt['Train %']/dt['2022 Q1 %'])
    dt['PSI'] = dt['Diff'] * dt['Log']
    dt['sum'] = dt['PSI'].sum()
    return dt